In [1]:
import time
import numpy as np
import pandas as pd
import os
import re
import datetime

# from dotenv import load_dotenv
from bs4 import BeautifulSoup
import urllib.request as req
import requests

In [2]:
os.getcwd()

'C:\\Users\\fuku_k\\Documents\\pypro\\vleague_stats'

In [10]:
path = '2021-22_regular/v1_w/teamdatabase.csv'
df = pd.read_csv(path)
df

,AA,AP,AE,ASucc%,AP/S,BAA,BAP,BAE,BASucc%,BP,...,OpBP/S,OpSVA,OpSVP,OpSVE,OpSVx,OpSVEff%,OpRA,OpRx,OpRg,OpRSucc%
0,158,63,16,39.9,15.75,14,4,0,28.6,6,...,2.50,93,1,10,27,5.6,91,47.0,10.0,57.1
1,158,56,15,35.4,14.00,10,1,1,10.0,10,...,1.50,97,4,5,31,10.8,82,39.0,16.0,57.3
2,152,44,14,28.9,11.00,22,2,4,9.1,9,...,2.00,95,5,9,23,8.9,78,43.0,13.0,63.5
3,149,61,12,40.9,15.25,22,10,4,45.5,8,...,2.25,86,3,7,21,7.6,85,41.0,18.0,58.8
4,94,52,5,55.3,17.33,11,6,1,54.5,7,...,0.67,47,0,2,12,5.3,65,15.0,23.0,40.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385,95,35,9,36.8,11.67,8,3,2,37.5,7,...,2.00,74,5,9,13,8.1,49,23.0,11.0,58.2
386,216,73,8,33.8,18.25,2,0,0,0.0,8,...,2.00,101,6,4,25,11.1,92,41.0,24.0,57.6
387,214,72,8,33.6,18.00,20,3,1,15.0,8,...,2.00,96,1,3,27,7.3,97,39.0,27.0,54.1
388,103,40,6,38.8,13.33,8,3,0,37.5,2,...,1.00,73,1,9,26,7.2,51,25.0,15.0,63.7


In [42]:
# df.iloc[:, 30:]
f = lambda x: 1 if x == 'Win' else 0
df['Win'] = df['Win Lose'].map(f)
df['Win']

0      1
1      0
2      0
3      1
4      1
      ..
385    0
386    0
387    1
388    0
389    1
Name: Win, Length: 390, dtype: int64

In [43]:
df.columns

Index(['AA', 'AP', 'AE', 'ASucc%', 'AP/S', 'BAA', 'BAP', 'BAE', 'BASucc%',
       'BP', 'BP/S', 'SVA', 'SVP', 'SVE', 'SVx', 'SVEff%', 'RA', 'Rx', 'Rg',
       'RSucc%', 'MatchNo.', 'Date', 'Year', 'Month', 'Day', 'Place', 'Venue',
       'Team', 'Op.Team', 'Director', 'Coach', 'Win Set', 'Lose Set',
       'Total Set', 'Win Lose', 'Get Point', 'Lose Point', 'Game Time',
       'Start Time', 'End Time', 'Spectators', 'Jury', 'Chief Umpire',
       'Sub Umpire', 'Judgeman', 'Home Away', 'OpAA', 'OpAP', 'OpAE',
       'OpASucc%', 'OpAP/S', 'OpBAA', 'OpBAP', 'OpBAE', 'OpBASucc%', 'OpBP',
       'OpBP/S', 'OpSVA', 'OpSVP', 'OpSVE', 'OpSVx', 'OpSVEff%', 'OpRA',
       'OpRx', 'OpRg', 'OpRSucc%', 'Win'],
      dtype='object')

In [59]:
df2 = df[['AA', 'AP', 'AE', 'ASucc%', 'AP/S', 'BAA', 'BAP', 'BAE', 'BASucc%',
       'BP', 'BP/S', 'SVA', 'SVP', 'SVE', 'SVx', 'SVEff%', 'RA', 'Rx', 'Rg',
       'RSucc%']]
df2['BASucc%'] = df2['BASucc%'].replace('-', 0).astype(float)

C:\Users\fuku_k\AppData\Local\Temp/ipykernel_7656/3469499047.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['BASucc%'] = df2['BASucc%'].replace('-', 0).astype(float)


In [60]:
df3 = df2.apply(lambda x: (x-x.mean())/x.std(), axis=0)
df3

,AA,AP,AE,ASucc%,AP/S,BAA,BAP,BAE,BASucc%,BP,BP/S,SVA,SVP,SVE,SVx,SVEff%,RA,Rx,Rg,RSucc%
0,0.297429,0.581084,2.369947,0.322460,0.381101,0.050672,-0.191553,-1.063493,-0.198829,-0.351680,-0.516818,0.817019,0.459516,-0.546748,1.253457,0.874532,0.394504,0.424470,-0.273493,-0.023393
1,0.297429,0.039110,2.072942,-0.441983,-0.398113,-0.440314,-1.067010,-0.301062,-1.238775,0.883954,0.736658,0.583028,-1.050692,1.167333,0.743310,-0.743376,0.941603,1.151333,-1.081539,-0.046727
2,0.125284,-0.889990,1.775937,-1.546179,-1.733907,1.032644,-0.775191,1.986229,-1.289095,0.575045,0.423289,0.173543,-0.043886,0.138885,-0.021910,-0.121104,0.576870,0.606186,-0.004144,0.151609
3,0.039211,0.426234,1.181927,0.492336,0.158469,1.032644,1.559362,1.986229,0.746069,0.266137,0.109920,0.700023,0.962919,0.824517,0.233163,0.283373,0.151349,0.787902,-0.677516,0.699949
4,-1.538789,-0.270590,-0.897107,2.938554,1.084620,-0.317567,0.392085,-0.301062,1.249270,-0.042772,0.523567,-0.528430,0.962919,0.138885,0.360700,1.403464,-1.854682,-1.574405,-0.273493,-0.221729
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385,-1.510098,-1.586814,0.290912,-0.204156,-1.435580,-0.685807,-0.483372,0.461368,0.298780,-0.042772,0.523567,-1.698385,-1.554094,-0.546748,-0.787130,-1.054513,-0.821272,-0.302394,-0.273493,0.629948
386,1.961501,1.355334,-0.006092,-0.713785,1.494263,-1.422285,-1.358829,-1.063493,-1.797887,0.266137,0.109920,0.758521,-1.050692,-1.232381,0.743310,-0.214445,1.306336,0.424470,1.207926,-0.396732
387,1.904120,1.277909,-0.006092,-0.747760,1.382947,0.787151,-0.483372,-0.301062,-0.959220,0.266137,0.109920,1.051010,1.466322,-0.889564,0.488237,0.967873,1.002392,0.606186,0.803903,0.011607
388,-1.280571,-1.199690,-0.600102,0.135596,-0.696440,-0.685807,-0.483372,-1.063493,0.298780,-1.587314,-1.557203,-1.405896,-0.547289,-0.203932,-1.297277,-0.774490,-0.699695,-1.029258,-0.542842,-1.236742


In [61]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 390 entries, 0 to 389
Data columns (total 20 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   AA       390 non-null    float64
 1   AP       390 non-null    float64
 2   AE       390 non-null    float64
 3   ASucc%   390 non-null    float64
 4   AP/S     390 non-null    float64
 5   BAA      390 non-null    float64
 6   BAP      390 non-null    float64
 7   BAE      390 non-null    float64
 8   BASucc%  390 non-null    float64
 9   BP       390 non-null    float64
 10  BP/S     390 non-null    float64
 11  SVA      390 non-null    float64
 12  SVP      390 non-null    float64
 13  SVE      390 non-null    float64
 14  SVx      390 non-null    float64
 15  SVEff%   390 non-null    float64
 16  RA       390 non-null    float64
 17  Rx       390 non-null    float64
 18  Rg       390 non-null    float64
 19  RSucc%   390 non-null    float64
dtypes: float64(20)
memory usage: 61.1 KB


In [62]:
import seaborn as sns
import matplotlib.pyplot as plt
# sns.pairplot(df2)
# plt.show()

In [69]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

X = df3 # 説明変数
Y = df['Win']
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0) # 80%のデータを学習データに、20%を検証データにする

lr = LogisticRegression() # ロジスティック回帰モデルのインスタンスを作成
lr_ = lr.fit(X_train, Y_train) # ロジスティック回帰モデルの重みを学習

In [74]:
lr.coef_[0]

array([ 0.67203229,  1.26701537,  0.10161815,  0.99612636,  0.40869425,
        0.24135511, -0.38887455,  0.11259394, -0.07634991,  0.06318933,
        0.58301615,  1.44530248,  0.55475007,  0.06594915, -0.15279674,
        0.07581114, -2.3469614 , -1.13475562, -0.64640073,  0.77583047])

In [65]:
df3.columns

Index(['AA', 'AP', 'AE', 'ASucc%', 'AP/S', 'BAA', 'BAP', 'BAE', 'BASucc%',
       'BP', 'BP/S', 'SVA', 'SVP', 'SVE', 'SVx', 'SVEff%', 'RA', 'Rx', 'Rg',
       'RSucc%'],
      dtype='object')

In [70]:
lr_.summary()

AttributeError: 'LogisticRegression' object has no attribute 'summary'

In [75]:
df4 = pd.DataFrame(list(zip(df3.columns, lr.coef_[0])), columns=['stats', 'coef_'])
df4

,stats,coef_
0,AA,0.672032
1,AP,1.267015
2,AE,0.101618
3,ASucc%,0.996126
4,AP/S,0.408694
5,BAA,0.241355
6,BAP,-0.388875
7,BAE,0.112594
8,BASucc%,-0.076350
9,BP,0.063189
